In [6]:
# Author: Dude123

import cv2
import os

# Function to split video into frames
def split_video_to_frames(video_path, output_dir):
    print(f"file exists: {os.path.isfile(video_path)}")
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print(ret, frame)
            print(f"finished processing video after {frame_count} frames")
            break
        frame_path = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1
    cap.release()
    print(f"Total frames extracted: {frame_count}")

# Function to perform blob detection
def detect_blobs(image, min_area=150, max_area=3000, min_circularity=0.1, min_convexity=0.5, min_inertia_ratio=0.01):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
     # Apply Gaussian blur
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    
    # Apply thresholding
    _, thresh_image = cv2.threshold(blurred_image, 60, 255, cv2.THRESH_BINARY_INV)
    
    # Set up the blob detector
    params = cv2.SimpleBlobDetector_Params()
    params.filterByArea = True
    params.minArea = min_area
    params.maxArea = max_area
    params.filterByCircularity = True
    params.minCircularity = min_circularity
    params.filterByConvexity = True
    params.minConvexity = min_convexity
    params.filterByInertia = True
    params.minInertiaRatio = min_inertia_ratio
    detector = cv2.SimpleBlobDetector_create(params)
    # Detect blobs
    keypoints = detector.detect(gray_image)
    return keypoints

# Function to save blobs as images
def save_blobs_as_images(frame_path, keypoints, output_dir):
    image = cv2.imread(frame_path)
    for i, kp in enumerate(keypoints):
        x = int(kp.pt[0] - kp.size / 2)
        y = int(kp.pt[1] - kp.size / 2)
        w = int(kp.size)
        h = int(kp.size)
        blob_roi = image[y:y+h, x:x+w]
        if blob_roi.size > 0:
            blob_path = os.path.join(output_dir, f"{os.path.basename(frame_path).split('.')[0]}_blob_{i:02d}.jpg")
            cv2.imwrite(blob_path, blob_roi)

# Paths
video_path_1 = "./videos/video4.avi"
video_path_2 = "./videos/video5.avi"
frames_output_dir = "Dataset/Frames"
blobs_output_dir = "Dataset/Blobs"

# Create output directories if they don't exist
os.makedirs(frames_output_dir, exist_ok=True)
os.makedirs(blobs_output_dir, exist_ok=True)

# Split videos into frames
split_video_to_frames(video_path_1, frames_output_dir)
split_video_to_frames(video_path_2, frames_output_dir)

# Process each frame for blob detection and save blobs
for frame_file in os.listdir(frames_output_dir):
    frame_path = os.path.join(frames_output_dir, frame_file)
    frame_image = cv2.imread(frame_path)
    keypoints = detect_blobs(frame_image)
    save_blobs_as_images(frame_path, keypoints, blobs_output_dir)


file exists: True
False None
finished processing video after 1877 frames
Total frames extracted: 1877
file exists: True
False None
finished processing video after 1251 frames
Total frames extracted: 1251
Processing completed.


In [ ]:
# Function to automatically label blobs
def label_blobs_automatically(blob_dir, label_output_file):
    labels = []
    for file_name in os.listdir(blob_dir):
        if file_name.endswith(".jpg"):
            blob_path = os.path.join(blob_dir, file_name)
            blob = cv2.imread(blob_path)
            height, width, _ = blob.shape
            center_x = width // 2
            size_threshold=50
            
            if center_x < width // 3:   
                label = "Left"
            elif center_x > 2 * width // 3:
                label = "Right"
            else:
                label = "None"
            
            labels.append((file_name, label))
    
    with open(label_output_file, 'w') as f:
        for file_name, label in labels:
            f.write(f"{file_name},{label}\n")


# Automatically label blobs
label_output_file = 'Dataset/blob_labels.csv'
label_blobs_automatically(blobs_output_dir, label_output_file)

print("Processing completed.")